In [1]:
pip install dask[dataframe]

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 241.9/241.9 kB 7.2 MB/s eta 0:00:00
     ------------------------------------- 242.8/242.8 kB 14.5 MB/s eta 0:00:00
     ------------------------------------- 241.7/241.7 kB 14.5 MB/s eta 0:00:00
     ------------------------------------- 206.6/206.6 kB 13.1 MB/s eta 0:00:00
     ------------------------------------- 205.9/205.9 kB 13.0 MB/s eta 0:00:00
     ------------------------------------- 205.9/205.9 kB 13.0 MB/s eta 0:00:00
     ------------------------------------- 205.7/205.7 kB 12.2 MB/s eta 0:00:00
     ------------------------------------- 205.5/205.5 kB 12.2 MB/s eta 0:00:00
     ------------------------------------- 205.4/205.4 kB 12.2 MB/s eta 0:00:00
     ------------------------------------- 205.1/205.1 kB 12.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
print("Vis setup Complete")
print("Setup complete.")

Vis setup Complete
Setup complete.


In [2]:
#Load the data using pandas read_csv function. This will generate a data frame we will use through the work.
cleaned_goalkeepers_df = pd.read_csv('goalkeepers_rating_2.csv')
cleaned_midfielders_df = pd.read_csv('midfielders_rating_2.csv')
cleaned_forwarders_df = pd.read_csv('forwarders_rating_2.csv')
cleaned_defenders_df = pd.read_csv('defenders_rating_2.csv')

In [3]:
#Show the first and last few records to check data is loading correctly
print(cleaned_forwarders_df.head())
print(cleaned_forwarders_df.tail())

  competition        date pos_role          player  goals  assists  \
0   Euro 2016  2016-06-10       FW  Olivier Giroud      1        0   
1   Euro 2016  2016-06-10       FW  Olivier Giroud      1        0   
2   Euro 2016  2016-06-10       FW  Olivier Giroud      1        0   
3   Euro 2016  2016-06-10      FWL   Dimitri Payet      1        1   
4   Euro 2016  2016-06-10      FWL   Dimitri Payet      1        1   

   shots_ontarget  shots_offtarget  shotsblocked  chances2score  ...  \
0               2                2             0              3  ...   
1               2                2             0              3  ...   
2               2                2             0              3  ...   
3               1                1             1              0  ...   
4               1                1             1              0  ...   

   missed_penalties  owngoals  degree_centrality  betweenness_centrality  \
0                 0         0           1.428571                0.3160

In [4]:
#shape
cleaned_forwarders_df.shape

(6666, 54)

In [5]:
#check the single values of the columns

In [6]:
import pandas as pd

df = cleaned_forwarders_df

# Iterate through each column and check for columns with a single unique value
single_value_columns = [column for column in df.columns if df[column].nunique() == 1]

# Print columns with a single unique value
print("Columns with a single unique value:")
for column in single_value_columns:
    print(column)

Columns with a single unique value:
goals_ag_otb
goals_ag_itb
saves_otb
saved_pen


In [7]:
#we dropped the columns, that has single value in the dataset and it is known that all goal leepers here played for 90 minutes as well

columns_to_drop = ['goals_ag_otb',
'goals_ag_itb',
'saves_itb',
'saves_otb',
'saved_pen']

# Dropping the columns and retaining the name goalkeepers_df
cleaned_forwarders_df = cleaned_forwarders_df.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned dataframe to verify
cleaned_forwarders_df.head()

,competition,date,pos_role,player,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,...,missed_penalties,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,minutesPlayed,game_duration
0,Euro 2016,2016-06-10,FW,Olivier Giroud,1,0,2,2,0,3,...,0,0,1.428571,0.316015,0.754464,0.315068,1.090213,0.050000,90,90
1,Euro 2016,2016-06-10,FW,Olivier Giroud,1,0,2,2,0,3,...,0,0,1.428571,0.316015,0.754464,0.315068,1.090213,0.050000,90,90
2,Euro 2016,2016-06-10,FW,Olivier Giroud,1,0,2,2,0,3,...,0,0,1.428571,0.316015,0.754464,0.315068,1.090213,0.050000,90,90
3,Euro 2016,2016-06-10,FWL,Dimitri Payet,1,1,1,1,1,0,...,0,0,1.428571,0.176156,0.754464,0.547945,1.465723,0.055556,90,90
4,Euro 2016,2016-06-10,FWL,Dimitri Payet,1,1,1,1,1,0,...,0,0,1.428571,0.176156,0.754464,0.547945,1.465723,0.055556,90,90


In [8]:
#number of players in each competition
# Group by competition and count the number of unique players in each competition
player_count_per_competition = cleaned_forwarders_df.groupby('competition')['player'].nunique().reset_index(name='unique_player_count')

# Display the results
player_count_per_competition

,competition,unique_player_count
0,Bundesliga 2017-18,132
1,Euro 2016,73
2,Premier League 2017-18,130
3,World Cup 2018,81


In [9]:
# Determine the first pos_role for each player in each competition
first_pos_role = cleaned_forwarders_df.groupby(['competition', 'player'])['pos_role'].first().reset_index()

# Merge this first pos_role back into the original dataframe
cleaned_forwarders_df = cleaned_forwarders_df.drop(columns=['pos_role'])
cleaned_forwarders_df = cleaned_forwarders_df.merge(first_pos_role, on=['competition', 'player'], how='left')

# Calculate the number of matches played per player in each competition
cleaned_forwarders_df['matches_played'] = cleaned_forwarders_df.groupby(['competition', 'player'])['date'].transform('nunique')

# Calculate the average network metrics, game duration, and minutes played
network_metrics = ['degree_centrality', 'betweenness_centrality', 'closeness_centrality', 'flow_centrality',  'betweenness2goals', 'game_duration','flow_success','minutesPlayed']
cleaned_forwarders_df[network_metrics] = cleaned_forwarders_df.groupby(['competition', 'player'])[network_metrics].transform('mean')

# Select only numeric columns for summing
columns_to_sum = cleaned_forwarders_df.select_dtypes(include='number').columns.difference(['matches_played'] + network_metrics)
cleaned_forwarders_df[columns_to_sum] = cleaned_forwarders_df.groupby(['competition', 'player'])[columns_to_sum].transform('sum')

# Drop duplicate rows and the date column
cleaned_forwarders_df = cleaned_forwarders_df.drop_duplicates(subset=['competition', 'player'])
cleaned_forwarders_df = cleaned_forwarders_df.drop(columns=['date'])

# Reorder columns to maintain the original order
ordered_columns = ['competition', 'pos_role', 'player', 'matches_played'] + list(columns_to_sum) + network_metrics
cleaned_forwarders_df = cleaned_forwarders_df[ordered_columns]

In [10]:
cleaned_forwarders_df.head()

,competition,pos_role,player,matches_played,aerials_l,aerials_w,assists,chances2score,clearances,countattack,...,wasfouled,ycards,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,flow_success,minutesPlayed
0,Euro 2016,FW,Olivier Giroud,5,46,122,6,29,7,7,...,13,3,1.158694,0.146642,0.679693,0.279271,0.072818,90.0,1.505637,75.823529
3,Euro 2016,FWL,Dimitri Payet,2,6,0,3,9,0,6,...,9,0,1.483516,0.181057,0.772837,0.521031,0.040936,90.0,1.564326,90.000000
6,Euro 2016,FW,Florin Andone,1,9,9,0,6,0,9,...,3,0,0.928571,0.133565,0.670635,0.173913,0.000000,90.0,0.867720,61.000000
9,Euro 2016,FWR,Antoine Griezmann,3,18,7,0,25,0,19,...,9,0,1.392308,0.156355,0.723349,0.365791,0.020000,90.0,1.707725,77.600000
12,Euro 2016,FW,Michal Duris,2,27,15,0,0,0,6,...,18,0,0.826531,0.069568,0.607948,0.130708,0.000000,90.0,0.687415,61.714286


In [11]:
#to check we have some players that played in other competitions
import pandas as pd


# Function to count the number of competitions for each player
def count_competitions_per_player(dataframe):
    return dataframe.groupby('player')['competition'].nunique().reset_index(name='Number of Competitions')

# Calculate the number of competitions for each player
player_competitions = count_competitions_per_player(cleaned_forwarders_df)

# Output the result
print(player_competitions)


                 player  Number of Competitions
0            Aaron Hunt                       1
1          Aaron Lennon                       1
2          Aaron Ramsey                       1
3          Adam Lallana                       1
4           Adam Szalai                       2
..                  ...                     ...
358      Yussuf Poulsen                       1
359          Yuya Osako                       2
360  Zlatan Ibrahimovic                       1
361    Zlatko Junuzovic                       1
362      Zoltan Stieber                       1

[363 rows x 2 columns]


In [12]:
#we dropped the columns, that has single value in the dataset and it is known that all goal leepers here played for 90 minutes as well

columns_to_drop = ['competition','player']

# Dropping the columns and retaining the name goalkeepers_df
cleaned_forwarders_df = cleaned_forwarders_df.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned dataframe to verify
cleaned_forwarders_df.head()

,pos_role,matches_played,aerials_l,aerials_w,assists,chances2score,clearances,countattack,crosses_acc,crosses_inacc,...,wasfouled,ycards,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,flow_success,minutesPlayed
0,FW,5,46,122,6,29,7,7,7,3,...,13,3,1.158694,0.146642,0.679693,0.279271,0.072818,90.0,1.505637,75.823529
3,FWL,2,6,0,3,9,0,6,24,27,...,9,0,1.483516,0.181057,0.772837,0.521031,0.040936,90.0,1.564326,90.000000
6,FW,1,9,9,0,6,0,9,0,0,...,3,0,0.928571,0.133565,0.670635,0.173913,0.000000,90.0,0.867720,61.000000
9,FWR,3,18,7,0,25,0,19,0,10,...,9,0,1.392308,0.156355,0.723349,0.365791,0.020000,90.0,1.707725,77.600000
12,FW,2,27,15,0,0,0,6,0,0,...,18,0,0.826531,0.069568,0.607948,0.130708,0.000000,90.0,0.687415,61.714286


In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import time

In [14]:
cleaned_forwarders_df.head()

,pos_role,matches_played,aerials_l,aerials_w,assists,chances2score,clearances,countattack,crosses_acc,crosses_inacc,...,wasfouled,ycards,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,flow_success,minutesPlayed
0,FW,5,46,122,6,29,7,7,7,3,...,13,3,1.158694,0.146642,0.679693,0.279271,0.072818,90.0,1.505637,75.823529
3,FWL,2,6,0,3,9,0,6,24,27,...,9,0,1.483516,0.181057,0.772837,0.521031,0.040936,90.0,1.564326,90.000000
6,FW,1,9,9,0,6,0,9,0,0,...,3,0,0.928571,0.133565,0.670635,0.173913,0.000000,90.0,0.867720,61.000000
9,FWR,3,18,7,0,25,0,19,0,10,...,9,0,1.392308,0.156355,0.723349,0.365791,0.020000,90.0,1.707725,77.600000
12,FW,2,27,15,0,0,0,6,0,0,...,18,0,0.826531,0.069568,0.607948,0.130708,0.000000,90.0,0.687415,61.714286


In [15]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
cleaned_forwarders_df.describe()

,matches_played,aerials_l,aerials_w,assists,chances2score,clearances,countattack,crosses_acc,crosses_inacc,dangmistakes,drib_success,drib_unsuccess,dribbled_past,fouls,goals,grduels_l,grduels_w,interceptions,keypasses,lballs_acc,lballs_inacc,missed_penalties,offsides,owngoals,passes_acc,passes_inacc,poss_lost,rcards,shots_offtarget,shots_ontarget,shotsblocked,stop_shots,tackles,tballs_acc,tballs_inacc,touches,wasfouled,ycards,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,flow_success,minutesPlayed
count,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000,416.000000
mean,6.247596,51.502404,35.848558,1.699519,21.944712,7.312500,26.543269,6.932692,15.362981,0.629808,13.084135,10.533654,4.391827,21.194712,5.110577,67.312500,73.550481,15.204327,5.817308,13.766827,12.721154,0.189904,9.447115,0.016827,271.259615,69.855769,164.024038,0.026442,18.286058,13.028846,6.718750,1.254808,9.293269,3.471154,7.615385,581.627404,21.610577,1.685096,1.103510,0.153068,0.669841,0.266449,0.019510,90.458076,0.936677,79.596026
std,7.395130,75.686544,62.962629,3.385098,29.433107,11.473319,35.242883,11.056712,22.203559,1.592899,20.038520,15.469171,6.023875,27.770888,9.223714,80.396148,90.699968,19.454218,8.328803,21.954958,16.033974,0.750899,14.700122,0.244861,339.729112,84.398329,188.547203,0.253691,24.747835,19.218067,9.820079,2.478335,12.701790,6.043149,11.275356,672.689411,27.277906,3.156843,0.231364,0.058495,0.064665,0.095041,0.025405,2.445033,0.403148,11.026923
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,0.000000,0.000000,0.285714,0.000000,0.464286,0.032258,0.000000,90.000000,0.000000,19.000000
25%,1.000000,8.000000,3.000000,0.000000,3.000000,0.000000,3.750000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,14.750000,15.000000,3.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,52.000000,15.000000,36.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,120.000000,3.000000,0.000000,0.981954,0.113825,0.635338,0.210678,0.000000,90.000000,0.694718,73.475000
50%,3.000000,20.000000,12.000000,0.000000,9.000000,3.000000,13.000000,3.000000,7.000000,0.000000,4.000000,4.000000,3.000000,12.000000,2.000000,36.000000,37.500000,8.000000,3.000000,6.000000,6.000000,0.000000,4.000000,0.000000,144.000000,36.000000,87.000000,0.000000,8.000000,6.000000,3.000000,0.000000,4.000000,0.000000,4.000000,323.000000,11.500000,0.000000,1.112424,0.153411,0.671903,0.252273,0.013031,90.000000,0.919876,82.057955
75%,7.000000,60.250000,36.500000,2.250000,29.000000,9.000000,34.000000,9.000000,20.000000,0.000000,17.250000,12.000000,6.000000,25.000000,6.000000,84.500000,92.250000,22.000000,8.000000,18.000000,17.000000,0.000000,12.000000,0.000000,346.000000,92.000000,206.000000,0.000000,24.000000,16.000000,9.000000,2.000000,12.000000,4.000000,10.000000,779.000000,29.250000,3.000000,1.233683,0.185080,0.707410,0.322514,0.029834,90.000000,1.159036,88.462121
max,37.000000,411.000000,488.000000,26.000000,210.000000,70.000000,242.000000,74.000000,166.000000,12.000000,148.000000,90.000000,41.000000,146.000000,68.000000,472.000000,572.000000,145.000000,54.000000,264.000000,116.000000,6.000000,104.000000,4.000000,2852.000000,585.000000,941.000000,3.000000,184.0

In [16]:
cleaned_forwarders_df.columns

Index(['pos_role', 'matches_played', 'aerials_l', 'aerials_w', 'assists',
       'chances2score', 'clearances', 'countattack', 'crosses_acc',
       'crosses_inacc', 'dangmistakes', 'drib_success', 'drib_unsuccess',
       'dribbled_past', 'fouls', 'goals', 'grduels_l', 'grduels_w',
       'interceptions', 'keypasses', 'lballs_acc', 'lballs_inacc',
       'missed_penalties', 'offsides', 'owngoals', 'passes_acc',
       'passes_inacc', 'poss_lost', 'rcards', 'shots_offtarget',
       'shots_ontarget', 'shotsblocked', 'stop_shots', 'tackles', 'tballs_acc',
       'tballs_inacc', 'touches', 'wasfouled', 'ycards', 'degree_centrality',
       'betweenness_centrality', 'closeness_centrality', 'flow_centrality',
       'betweenness2goals', 'game_duration', 'flow_success', 'minutesPlayed'],
      dtype='object')

In [17]:
cleaned_forwarders_df.goals.value_counts()

goals
0     206
4      37
3      29
2      28
6      25
8      12
12     11
10      7
9       6
16      6
15      6
24      4
20      3
22      3
32      3
26      3
18      2
14      2
27      2
5       2
30      2
29      2
11      2
68      1
37      1
38      1
58      1
21      1
31      1
57      1
35      1
42      1
40      1
7       1
36      1
13      1
Name: count, dtype: int64

In [18]:
# lets move towards splitting the data.

# get the locations
y = cleaned_forwarders_df['goals']
X = cleaned_forwarders_df.drop(columns=['goals'], axis=1)

In [19]:
X.head()

,pos_role,matches_played,aerials_l,aerials_w,assists,chances2score,clearances,countattack,crosses_acc,crosses_inacc,dangmistakes,drib_success,drib_unsuccess,dribbled_past,fouls,grduels_l,grduels_w,interceptions,keypasses,lballs_acc,lballs_inacc,missed_penalties,offsides,owngoals,passes_acc,passes_inacc,poss_lost,rcards,shots_offtarget,shots_ontarget,shotsblocked,stop_shots,tackles,tballs_acc,tballs_inacc,touches,wasfouled,ycards,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,flow_success,minutesPlayed
0,FW,5,46,122,6,29,7,7,7,3,0,0,0,0,21,44,36,23,3,4,7,0,12,0,202,87,163,0,30,12,8,4,10,3,7,585,13,3,1.158694,0.146642,0.679693,0.279271,0.072818,90.0,1.505637,75.823529
3,FWL,2,6,0,3,9,0,6,24,27,3,18,3,3,0,27,69,6,18,36,21,0,0,0,282,69,129,0,15,6,9,0,3,3,12,537,9,0,1.483516,0.181057,0.772837,0.521031,0.040936,90.0,1.564326,90.000000
6,FW,1,9,9,0,6,0,9,0,0,0,3,6,0,9,30,6,0,0,0,0,0,0,0,18,3,36,0,6,0,0,0,0,0,0,63,3,0,0.928571,0.133565,0.670635,0.173913,0.000000,90.0,0.867720,61.000000
9,FWR,3,18,7,0,25,0,19,0,10,0,6,0,3,7,29,22,6,8,10,14,0,3,0,317,65,120,0,18,19,9,3,7,0,7,505,9,0,1.392308,0.156355,0.723349,0.365791,0.020000,90.0,1.707725,77.600000
12,FW,2,27,15,0,0,0,6,0,0,0,0,4,0,22,47,22,3,0,0,0,0,11,0,50,7,47,0,4,0,4,0,3,0,0,105,18,0,0.826531,0.069568,0.607948,0.130708,0.000000,90.0,0.687415,61.714286


In [20]:
y.head()

0     9
3     3
6     0
9     6
12    0
Name: goals, dtype: int64

In [21]:
X.columns

Index(['pos_role', 'matches_played', 'aerials_l', 'aerials_w', 'assists',
       'chances2score', 'clearances', 'countattack', 'crosses_acc',
       'crosses_inacc', 'dangmistakes', 'drib_success', 'drib_unsuccess',
       'dribbled_past', 'fouls', 'grduels_l', 'grduels_w', 'interceptions',
       'keypasses', 'lballs_acc', 'lballs_inacc', 'missed_penalties',
       'offsides', 'owngoals', 'passes_acc', 'passes_inacc', 'poss_lost',
       'rcards', 'shots_offtarget', 'shots_ontarget', 'shotsblocked',
       'stop_shots', 'tackles', 'tballs_acc', 'tballs_inacc', 'touches',
       'wasfouled', 'ycards', 'degree_centrality', 'betweenness_centrality',
       'closeness_centrality', 'flow_centrality', 'betweenness2goals',
       'game_duration', 'flow_success', 'minutesPlayed'],
      dtype='object')

In [22]:
X.dtypes

pos_role                   object
matches_played              int64
aerials_l                   int64
aerials_w                   int64
assists                     int64
chances2score               int64
clearances                  int64
countattack                 int64
crosses_acc                 int64
crosses_inacc               int64
dangmistakes                int64
drib_success                int64
drib_unsuccess              int64
dribbled_past               int64
fouls                       int64
grduels_l                   int64
grduels_w                   int64
interceptions               int64
keypasses                   int64
lballs_acc                  int64
lballs_inacc                int64
missed_penalties            int64
offsides                    int64
owngoals                    int64
passes_acc                  int64
passes_inacc                int64
poss_lost                   int64
rcards                      int64
shots_offtarget             int64
shots_ontarget

In [23]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

numeric_transformer = MinMaxScaler()  
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)
X = preprocessor.fit_transform(X)
# Get the numerical column names after scaling
num_col_names = num_features

# Get the categorical column names after one hot encoding
cat_col_names = preprocessor.named_transformers_['OneHotEncoder'].get_feature_names_out(cat_features)

# Combine all column names
all_col_names = list(num_col_names) + list(cat_col_names)
X= pd.DataFrame(X, columns=all_col_names)
print(X.head())

   matches_played  aerials_l  aerials_w   assists  chances2score  clearances  \
0             1.0        0.0        0.0  0.111111       0.111922    0.250000   
1             0.0        1.0        0.0  0.027778       0.014599    0.000000   
2             1.0        0.0        0.0  0.000000       0.021898    0.018443   
3             0.0        0.0        1.0  0.055556       0.043796    0.014344   
4             1.0        0.0        0.0  0.027778       0.065693    0.030738   

   countattack  crosses_acc  crosses_inacc  dangmistakes  drib_success  \
0     0.230769     0.138095            0.1      0.028926      0.094595   
1     0.115385     0.042857            0.0      0.024793      0.324324   
2     0.000000     0.028571            0.0      0.037190      0.000000   
3     0.000000     0.119048            0.0      0.078512      0.000000   
4     0.000000     0.000000            0.0      0.024793      0.000000   

   drib_unsuccess  dribbled_past     fouls  grduels_l  grduels_w  \
0     

In [24]:
X.shape

(416, 48)

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now, split the train+validation set into separate training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Display the sizes of the splits to ensure they are correct
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 249
Validation set size: 83
Test set size: 84


In [26]:
X_train.head()

,matches_played,aerials_l,aerials_w,assists,chances2score,clearances,countattack,crosses_acc,crosses_inacc,dangmistakes,drib_success,drib_unsuccess,dribbled_past,fouls,grduels_l,grduels_w,interceptions,keypasses,lballs_acc,lballs_inacc,missed_penalties,offsides,owngoals,passes_acc,passes_inacc,poss_lost,rcards,shots_offtarget,shots_ontarget,shotsblocked,stop_shots,tackles,tballs_acc,tballs_inacc,touches,wasfouled,ycards,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,flow_success,minutesPlayed,pos_role_FW,pos_role_FWL,pos_role_FWR
297,0.0,0.0,1.0,0.111111,0.058394,0.000000,0.000000,0.047619,0.142857,0.082645,0.162162,0.192771,0.000000,0.135135,0.088889,0.097561,0.027397,0.093220,0.111888,0.124138,0.037037,0.045455,0.086207,0.0,0.038462,0.0,0.063904,0.085470,0.132620,0.0,0.054348,0.043478,0.0750,0.000000,0.119048,0.000000,0.055556,0.095656,0.138889,0.083333,0.589841,0.470816,0.465722,0.401261,0.098086,0.0,0.328265,0.645545
257,0.0,0.0,1.0,0.000000,0.014599,0.006148,0.000000,0.000000,0.042857,0.012397,0.000000,0.000000,0.000000,0.000000,0.000000,0.073171,0.020548,0.038136,0.020979,0.020690,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.020716,0.005128,0.012834,0.0,0.016304,0.000000,0.0375,0.214286,0.071429,0.071429,0.000000,0.018946,0.020833,0.000000,0.619048,0.465078,0.368421,0.277969,0.000000,0.0,0.226351,0.623762
151,0.0,0.0,1.0,0.111111,0.051095,0.018443,0.000000,0.114286,0.042857,0.086777,0.000000,0.018072,0.000000,0.060811,0.200000,0.219512,0.184932,0.158898,0.141608,0.041379,0.055556,0.045455,0.077586,0.0,0.086538,0.0,0.051264,0.046154,0.105882,0.0,0.163043,0.043478,0.1125,0.000000,0.142857,0.000000,0.041667,0.079945,0.208333,0.125000,0.436825,0.542624,0.335979,0.283772,0.133929,0.0,0.484803,0.500990
374,1.0,0.0,0.0,0.055556,0.136253,0.024590,0.153846,0.114286,0.114286,0.049587,0.054054,0.144578,0.333333,0.081081,0.133333,0.000000,0.164384,0.101695,0.111888,0.110345,0.074074,0.060606,0.206897,0.0,0.076923,0.0,0.071629,0.102564,0.168984,0.0,0.065217,0.144928,0.0000,0.000000,0.142857,0.190476,0.222222,0.109519,0.111111,0.333333,0.587302,0.529789,0.507611,0.399286,0.075758,0.0,0.356151,0.627063
230,1.0,0.0,0.0,0.222222,0.255474,0.190574,0.000000,0.300000,0.471429,0.123967,0.040541,0.090361,0.000000,0.040541,0.100000,0.219512,0.123288,0.197034,0.104895,0.186207,0.111111,0.181818,0.310345,0.0,0.173077,0.0,0.152388,0.215385,0.266310,0.0,0.228261,0.239130,0.1875,0.000000,0.071429,0.000000,0.166667,0.213031,0.166667,0.000000,0.592945,0.549112,0.469113,0.334815,0.105820,0.0,0.408525,0.666667


In [27]:
# Save each DataFrame as a CSV file
X.to_csv('forwarders_rating_saves_X.csv', index=False)
y.to_csv('forwarders_rating_saves_y.csv', index=False)


print("DataFrames have been saved as CSV files.")

DataFrames have been saved as CSV files.
